# Population density

## Download data

Source: https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/population-distribution-demography/geostat

In [ ]:
from urllib import request
from os import path

data_dir = 'data'
url = 'http://ec.europa.eu/eurostat/cache/GISCO/geodatafiles/GEOSTAT-grid-POP-1K-2011-V2-0-1.zip'
dst = path.join(data_dir, 'compressed', 'popdensity.zip')
res = request.urlopen(url)
with open(dst, 'wb') as f:
    f.write(res.read())

## Transform raw data

The format of the source data is ETRS89/LAEA in 1km squares, where the data point is the bottom left corner of the square. Note that the points are transformed from km offset to m offset

In [ ]:
from os import path
import pandas as pd
import numpy as np
import re
from pyproj import CRS, Transformer

data_dir = 'data'
src_name = path.join(data_dir, 'popdensity_raw.csv')
dst_name = path.join(data_dir, 'popdensity.csv')

crs_src = CRS.from_string('EPSG:3035')
crs_dst = CRS.from_string('EPSG:4326')
trans = Transformer.from_crs(crs_src,crs_dst)

if not path.exists(dst_name):
    df = pd.read_csv(src_name)
    df.drop(['CNTR_CODE','METHD_CL','YEAR','DATA_SRC','TOT_P_CON_DT'], axis='columns', inplace=True)
    pop = np.empty(len(df.index), np.int32)
    lats = np.empty(len(df.index), np.float32)
    longs = np.empty(len(df.index), np.float32)
    for i, row in df.iterrows():
        s = row['GRD_ID']
        n = int(s[-9:-5]) * 1000 # km to m
        e = int(s[-4:]) * 1000
        lat, lon = trans.transform(n, e)
        pop[i] = np.int32(row['TOT_P'])
        lats[i] = np.float32(lat)
        longs[i] = np.float32(lon)
    res = pd.DataFrame({
        'pop': pop,
        'lat': lats,
        'lon': longs
    })
    res.to_csv(dst_name, index=False)
else:
    print('Transformed data already exists!')

## Mapping the data between formats

`to_latlon` and `from_latlon` can be used to map between the source format and latitude/longitude.

In [ ]:
from pyproj import CRS, Transformer

crs_etrs = CRS.from_string('EPSG:3035')
crs_latlon = CRS.from_string('EPSG:4326')
transformer_to_latlon = Transformer.from_crs(crs_etrs, crs_latlon)
transformer_from_latlon = Transformer.from_crs(crs_latlon, crs_etrs)

def to_latlon(north, east): return transformer_to_latlon.transform(north, east)
def from_latlon(lat, lon): return transformer_from_latlon.transform(lat, lon)

to_latlon(*from_latlon(5,5))